<a href="https://colab.research.google.com/github/shimo8810/machine_intelligence/blob/master/deep_learning/colab/optuna_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!curl https://colab.chainer.org/install | sh -
!pip install -q optuna

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1379  100  1379    0     0  11491      0 --:--:-- --:--:-- --:--:-- 11491
+ apt -y -q install cuda-libraries-dev-9-2
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-9-2 is already the newest version (9.2.148-1).
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.
+ pip install -q cupy-cuda92  chainer 
+ set +ex
Installation succeeded!


In [0]:
import numpy as np
import chainer
import chainer.functions as F
import chainer.links as L
import optuna

In [0]:
N_TRAIN_EXAMPLES = 3000
N_TEST_EXAMPLES = 1000
BATCHSIZE = 128
EPOCH = 10

## model

In [0]:
class MLP(chainer.Chain):

    def __init__(self, n_out, n_units, n_layers, activation=F.sigmoid):
        self.activation = activation
        super(MLP, self).__init__()
        with self.init_scope():
            self.fc_in = L.Linear(None, n_units)

            self._forward = []
            for i in range(n_layers):
                name = 'fc_{}'.format(i)
                setattr(self, name, L.Linear(None, n_units))
                self._forward.append(name)

            self.fc_out = L.Linear(None, n_out)
            
    def forward(self, x):
        h = self.activation(self.fc_in(x))
        
        for name in self._forward:
            l = getattr(self, name)
            h = self.activation(l(h))
            
        return self.fc_out(h)

def create_model(trial):
    n_units = int(trial.suggest_discrete_uniform('n_units', 10, 100, 10))
    n_layers = trial.suggest_int('n_layers', 1, 4)
    activation_name = trial.suggest_categorical('activation',
                        ['sigmoid', 'relu', 'tanh'])
    activation = getattr(F, activation_name)
    
    return MLP(10, n_units, n_layers, activation)

## optimizer

In [0]:
def create_optimizer(trial, model):
    optimizer_name = trial.suggest_categorical('optimizer',
                     ['Adam', 'MomentumSGD'])
    if optimizer_name == 'Adam':
        adam_alpha = trial.suggest_loguniform('adam_alpha', 1e-5, 1e-1)
        optimizer = chainer.optimizers.Adam(alpha=adam_alpha)
    else:
        momentum_sgd_lr = trial.suggest_loguniform('momentum_sgd_lr', 1e-5, 1e-1)
        optimizer = chainer.optimizers.MomentumSGD(lr=momentum_sgd_lr)

    weight_decay = trial.suggest_loguniform('weight_decay', 1e-10, 1e-3)
    optimizer.setup(model)
    optimizer.add_hook(chainer.optimizer.WeightDecay(weight_decay))
    return optimizer

## objective

In [0]:
def objective(trial):
    # Model and optimizer
    model = L.Classifier(create_model(trial))
    optimizer = create_optimizer(trial, model)

    # Dataset
    rng = np.random.RandomState(0)
    train, test = chainer.datasets.get_mnist()
    train = chainer.datasets.SubDataset(
        train, 0, N_TRAIN_EXAMPLES, order=rng.permutation(len(train)))
    test = chainer.datasets.SubDataset(
        test, 0, N_TEST_EXAMPLES, order=rng.permutation(len(test)))
    train_iter = chainer.iterators.SerialIterator(train, BATCHSIZE)
    test_iter = chainer.iterators.SerialIterator(test, BATCHSIZE, repeat=False, shuffle=False)
    
    # Trainer
    updater = chainer.training.StandardUpdater(train_iter, optimizer)
    trainer = chainer.training.Trainer(updater, (EPOCH, 'epoch'))
    trainer.extend(chainer.training.extensions.Evaluator(test_iter, model))
    log_report_extension = chainer.training.extensions.LogReport(log_name=None)
    trainer.extend(chainer.training.extensions.PrintReport(
        ['epoch', 'main/loss', 'validation/main/loss',
         'main/accuracy', 'validation/main/accuracy']))
    trainer.extend(log_report_extension)

    # Run!
    trainer.run()
    
    # Set the user attributes such as loss and accuracy for train and validation sets
    log_last = log_report_extension.log[-1]
    for key, value in log_last.items():
        trial.set_user_attr(key, value)

    # Return the validation error
    val_err = 1.0 - log_report_extension.log[-1]['validation/main/accuracy']
    return val_err

## optimize

In [0]:
study = optuna.create_study()
study.optimize(objective, n_trials=100)

print('Number of finished trials: ', len(study.trials))

print('Best trial:')
trial = study.best_trial

print('  Value: ', trial.value)

print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

print('  User attrs:')
for key, value in trial.user_attrs.items():
    print('    {}: {}'.format(key, value))

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy
1           2.07738     1.79547               0.356771       0.646184                  
2           1.62434     1.43707               0.700747       0.764648                  
3           1.32127     1.18216               0.775065       0.810472                  
4           1.1118      1.00403               0.820992       0.83774                   
5           0.952693    0.869514              0.843099       0.860352                  
6           0.827092    0.767789              0.856997       0.86591                   
7           0.735396    0.688137              0.869141       0.877554                  
8           0.657666    0.628023              0.878736       0.882662                  
9           0.595927    0.574053              0.890965       0.88732                   


[I 2018-12-03 17:40:26,246] Finished a trial resulted in value: 0.10975059866905212. Current best value is 0.10975059866905212 with parameters: {'n_units': 30.0, 'n_layers': 1, 'activation': 'tanh', 'optimizer': 'Adam', 'adam_alpha': 0.0005071164751030867, 'weight_decay': 9.379027769101591e-06}.


epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy
1           2.28589     2.2558                0.148112       0.270808                  
2           2.19905     2.10375               0.317935       0.348032                  
3           1.93406     1.70643               0.398438       0.461238                  
4           1.44943     1.13387               0.523438       0.652794                  
5           0.951819    0.819519              0.695638       0.707632                  
6           0.719757    0.636095              0.764266       0.803861                  
7           0.579517    0.558633              0.823242       0.838642                  
8           0.496562    0.518161              0.844769       0.844501                  
9           0.419396    0.480121              0.873641       0.864258                  


[I 2018-12-03 17:40:29,435] Finished a trial resulted in value: 0.12454927712678909. Current best value is 0.10975059866905212 with parameters: {'n_units': 30.0, 'n_layers': 1, 'activation': 'tanh', 'optimizer': 'Adam', 'adam_alpha': 0.0005071164751030867, 'weight_decay': 9.379027769101591e-06}.


epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy
1           0.826892    0.402956              0.739258       0.888146                  
2           0.334832    0.380677              0.901155       0.888897                  
3           0.238626    0.440023              0.930664       0.857647                  
4           0.191056    0.352075              0.93716        0.90595                   
5           0.144518    0.348676              0.955078       0.894982                  
6           0.113083    0.37522               0.961957       0.902494                  
7           0.099311    0.355573              0.968099       0.911283                  
8           0.0788153   0.404464              0.974185       0.90933                   
9           0.0943414   0.472901              0.966712       0.880334                  


...

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy
1           2.09862     2.30803               0.336914       0.347806                  
2           1.56781     0.966816              0.441916       0.669847                  
3           0.738276    0.815791              0.763021       0.722882                  
4           0.587353    0.488402              0.815557       0.868389                  
5           0.328216    0.35512               0.907227       0.904447                  
6           0.247592    0.401255              0.926291       0.891076                  
7           0.195314    0.276736              0.942057       0.927809                  
8           0.136644    0.336346              0.962976       0.910983                  
9           0.116703    0.333028              0.962296       0.912184                  


[I 2018-12-03 17:46:47,978] Finished a trial resulted in value: 0.10802283883094788. Current best value is 0.06798377633094788 with parameters: {'n_units': 100.0, 'n_layers': 4, 'activation': 'relu', 'optimizer': 'MomentumSGD', 'momentum_sgd_lr': 0.06468383425047516, 'weight_decay': 0.0007499328180966753}.


epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy
1           1.94001     0.984649              0.38151        0.667142                  
2           1.02403     0.634436              0.69803        0.829177                  
3           0.456939    0.377593              0.859701       0.886418                  
4           0.282202    0.306353              0.912024       0.905649                  
5           0.193458    0.328901              0.934896       0.892954                  
6           0.155124    0.251885              0.949728       0.91857                   
7           0.0851358   0.334499              0.974935       0.900691                  
8           0.0923132   0.293499              0.96875        0.909555                  
9           0.0775295   0.334553              0.975883       0.914964                  


[I 2018-12-03 17:46:51,770] Finished a trial resulted in value: 0.08383413404226303. Current best value is 0.06798377633094788 with parameters: {'n_units': 100.0, 'n_layers': 4, 'activation': 'relu', 'optimizer': 'MomentumSGD', 'momentum_sgd_lr': 0.06468383425047516, 'weight_decay': 0.0007499328180966753}.


Number of finished trials:  100
Best trial:
  Value:  0.06798377633094788
  Params: 
    n_units: 100.0
    n_layers: 4
    activation: relu
    optimizer: MomentumSGD
    momentum_sgd_lr: 0.06468383425047516
    weight_decay: 0.0007499328180966753
  User attrs:
    main/loss: 0.059863834253822766
    main/accuracy: 0.9801432291666666
    validation/main/loss: 0.30149781703948975
    validation/main/accuracy: 0.9320162236690521
    epoch: 10
    iteration: 235
    elapsed_time: 3.6612681579999844
